In [41]:
import models
import pandas as pd

In [42]:
filters = 32
kernels = 5
units = 128

In [43]:
CNN = f'CNN_{filters}_filters_{kernels}_kernels_predictions'
CNNA = f'CNN_Attention_{filters}_filters_{kernels}_kernels_predictions'
RNN = f'RNN_{units}_units_predictions'
LSTM = f'LSTM_{units}_layers_predictions'

In [44]:
MODEL = CNNA
COMMODITY = 'nickel_no_val_20'
RULE_NUM = 20
confidence_levels = [0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

algo = ['correction', 'detection_correction']
ALGO = algo[1]

In [45]:
# df = models.npy_to_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'copper/test/predictions/test_{METRIC}_{ALGO}/', confidence_levels)
df = models.npy_to_top_n_f1_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}.csv', f'{COMMODITY}/test/predictions/test_F1_{ALGO}', RULE_NUM)
df

,pred,corr,true_positive,false_positive,rule0,rule1,rule2,rule3,rule4,rule5,...,rule10,rule11,rule12,rule13,rule14,rule15,rule16,rule17,rule18,rule19
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
def calculate_metrics(df, rule_result_column):
    TP = ((df[rule_result_column] == 1) & (df['corr'] == 1)).sum()
    FP = ((df[rule_result_column] == 1) & (df['corr'] == 0)).sum()
    FN = ((df[rule_result_column] == 0) & (df['corr'] == 1)).sum()
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, recall, precision

In [47]:
f1, recall, precision = calculate_metrics(df, "pred")
print(f'Base model results\nf1: {f1}\nrecall: {recall}\nprecision: {precision}')

Base model results
f1: 0.23463687150837986
recall: 0.3333333333333333
precision: 0.1810344827586207


In [48]:
df.to_numpy().dump('edcr/data/test.npy')